In [1]:
import duckdb
from sams.config import RAW_DATA_DIR

2025-11-06 09:35:38.365 | INFO     | sams.config:<module>:15 - PROJ_ROOT path is: /Users/ymohanty/Documents/GitHub/sams
2025-11-06 09:35:38.378 | WARNING  | sams.config:<module>:83 - Google MAPS API key not found, using Nominatim geocoder
2025-11-06 09:35:38.388 | INFO     | sams.config:<module>:92 - Loaded 61900 geocodes from cache


# Ingredients of the unique key 
- Name
- Roll no
- DOB
- Passing year
- Exam board 
- Exam type

In [ ]:
# Connect to the database
conn = duckdb.connect()

conn.execute("INSTALL sqlite_scanner;")
conn.execute("LOAD sqlite_scanner;")

conn.execute(f"ATTACH '{RAW_DATA_DIR / 'sams.db'}' as sams (TYPE sqlite);")




In [33]:
# Examine mark data in each module
query = """
        SELECT DISTINCT on (module)
            module,
            mark_data
        FROM sams.students
        ORDER BY module, mark_data;
        """
conn.sql(query).show()



┌─────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ module  │                                                                                                                                                                                                                                                                                  mark_data                                                                                                                                        

In [ ]:
# Construct ingredients of the unique key 
query = """
            SELECT 
                module,
                academic_year,
                aadhar_no,
                dob,
                student_name,
                reported_institute,
                (json_extract(u.mark_data, '$.ExamName'))::VARCHAR as exam_name,
                (json_extract(u.mark_data, '$.YearofPassing'))::INT as passing_year,
                (json_extract(u.mark_data, '$.HighestQualificationExamBoard'))::VARCHAR as exam_board,
                (json_extract(u.mark_data, '$.ExaminationType'))::VARCHAR as exam_type,
                (json_extract(u.mark_data, '$.RollNo'))::VARCHAR as roll_no
            FROM sams.students as s 
            LEFT JOIN  UNNEST(
                coalesce(from_json(s.mark_data, '["JSON"]'), array[]::JSON[])
            ) as u(mark_data)
            ON TRUE
            WHERE s.module = 'ITI' or s.module = 'Diploma' or s.module = 'PDIS';
        """
unique_key_ingredients = conn.sql(query)

ParserException: Parser Error: syntax error at or near "LEFT"

LINE 16:             LEFT JOIN  UNNEST(
                     ^

In [ ]:

# Count null values for each column in unique_key_ingredients
columns = [desc[0] for desc in unique_key_ingredients.description]
null_counts_query = "SELECT " + ", ".join([f"COUNT(*) - COUNT({col}) as {col}_nulls" for col in columns]) + " FROM unique_key_ingredients"
null_counts_df = conn.sql(null_counts_query).df()
null_counts_df.T  # Transpose for readability


,0
module_nulls,0
academic_year_nulls,0
aadhar_no_nulls,85000
dob_nulls,0
student_name_nulls,0
reported_institute_nulls,6692645
exam_name_nulls,6322309
passing_year_nulls,6322309
exam_board_nulls,6322309
exam_type_nulls,6322309


In [28]:
# Construct the unique key 
query = """
            SELECT
                module,
                academic_year,
                aadhar_no,
                student_name || '-' || dob || '-' || passing_year || '-' || exam_board || '-' || exam_type || '-' || roll_no as unique_key
            FROM unique_key_ingredients
        """

unique_key_rel = conn.sql(query)

In [31]:
query = """
    SELECT COUNT(DISTINCT aadhar_no), COUNT(DISTINCT unique_key) from unique_key_rel
    """
conn.sql(query).show()

┌───────────────────────────┬────────────────────────────┐
│ count(DISTINCT aadhar_no) │ count(DISTINCT unique_key) │
│           int64           │           int64            │
├───────────────────────────┼────────────────────────────┤
│                   4133537 │                    1015322 │
└───────────────────────────┴────────────────────────────┘



In [29]:
query = """
SELECT academic_year, COUNT(DISTINCT aadhar_no), COUNT(DISTINCT unique_key) from unique_key_rel
GROUP BY academic_year
ORDER BY academic_year
"""
conn.sql(query).show()


┌───────────────┬───────────────────────────┬────────────────────────────┐
│ academic_year │ count(DISTINCT aadhar_no) │ count(DISTINCT unique_key) │
│     int64     │           int64           │           int64            │
├───────────────┼───────────────────────────┼────────────────────────────┤
│          2017 │                     27337 │                      30446 │
│          2018 │                    691492 │                     142541 │
│          2019 │                    660721 │                     126939 │
│          2020 │                    658449 │                     131300 │
│          2021 │                    750153 │                     134791 │
│          2022 │                    811347 │                     166020 │
│          2023 │                    821893 │                     184090 │
│          2024 │                    895342 │                     176920 │
└───────────────┴───────────────────────────┴────────────────────────────┘



In [ ]:
query = """
SELECT COUNT(*) from unique_key_rel
"""
conn.sql(query).show()


In [18]:
query = """
WITH temp_table_1 as (
    SELECT 
        unique_key,
        CASE WHEN SUM(CASE WHEN module = 'ITI' THEN 1 ELSE 0 END) > 0 THEN 1 ELSE 0 END as applied_iti,
        CASE WHEN SUM(CASE WHEN module = 'Diploma' THEN 1 ELSE 0 END) > 0 THEN 1 ELSE 0 END as applied_diploma,
        CASE WHEN SUM(CASE WHEN module = 'HSS' THEN 1 ELSE 0 END) > 0 THEN 1 ELSE 0 END as applied_hss,
        CASE WHEN SUM(CASE WHEN module = 'DEG' THEN 1 ELSE 0 END) > 0 THEN 1 ELSE 0 END as applied_degree,
    FROM unique_key_rel
    GROUP BY unique_key
)
SELECT 
    unique_key,
    applied_iti,
    applied_diploma,
    applied_hss,
    applied_degree,
    applied_iti + applied_diploma + applied_hss + applied_degree as total_applied
FROM temp_table_1

"""
multiple_modules = conn.sql(query)






In [ ]:
multiple_modules.show()

In [24]:
query = """
SELECT COUNT(*) from multiple_modules
where applied_diploma = 1 and applied_hss = 1
"""
conn.sql(query).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│            0 │
└──────────────┘

